In [1]:
#--------------- Librerias ----------------------------------------------
#Librerias tipicas para el análisis de datos
import os
import warnings
import numpy as np
import time
from datetime import datetime
warnings.filterwarnings('ignore')

#Librerias para la implementación de pyspark
os.environ["HADOOP_HOME"] = "C:\\hadoop"
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#Libreraias para la implementación de keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
#----------------------------------------------------------------------------

In [4]:
#-----------------------SparkSession-----------------------------------------
spark = SparkSession.builder \
    .appName("NYC_Taxi_Spark_DeepLearning") \
    .master("local[8]") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.python.worker.timeout", "600s") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.default.parallelism", "16") \
    .config("spark.rdd.compress", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

# appName: Nombre de la aplicación de spark 
# master, local[8] ejecuta localmente con 8 threads (uno por core)
# driver.memory 12g: memoria ram para el driver, mi computadora tiene 16gb de ram
# executor.memory 12: memoria ram para los workers 
# spark.driver.maxResultSize: Máximo tamaño de resultados que collect() puede traer
# .config("spark.network.timeout", "800s") Timeout para comunicación entre nodos, por default 120s (2 minutos), mi ajuste  800s (13 minutos) para evitar timeouts con datos grandes.
# .config("spark.executor.heartbeatInterval", "60s"): Frecuencia con que ejecutores reportan al driver que siguen vivos. Default: 10s. Mi ajuste : 60s reduce overhead de comunicación.
# .config("spark.python.worker.timeout", "600s"): Timeout específico para workers de Python. Default: 120s. MI ajuste: 600s (10 minutos) para operaciones Python lentas.
# .config("spark.default.parallelism", "16"): Paralelismo por defecto para operaciones RDD. Mi ajuste: 16 particiones = 2 por core. 
# config("spark.rdd.compress", "true"): Comprimir RDDs serializados en memoria. Ventaja: Usa menos RAM (importante para cache). Costo: CPU extra para comprimir/descomprimir. 
# .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer"): Protocolo de serialización de objetos. Default: Java serialization (lento). Kryo: 10x más rápido y usa menos memoria.

print(" Sessión de Spark creada")
print("  Network timeout: 800s")
print("  Worker timeout: 600s")

 Sessión de Spark creada
  Network timeout: 800s
  Worker timeout: 600s


In [8]:
#----------------------Cargar datos-----------------------------------------------
DATA_PATH = "C:/Users/PC/Documents/DocumentosGustavo/Github/Maestria/BigData/nyc-taxi-spark/data/yellow/2024/yellow_tripdata_2024-01.parquet"

print("\n Cargando dataset")
df = spark.read.parquet(DATA_PATH)
print(f" Numero de registros: {df.count():,}")
print(f" Columnas: {len(df.columns)}")


 Cargando dataset
 Numero de registros: 2,964,624
 Columnas: 19


In [10]:
print("\nEsquema del dataset:")
df.printSchema()


Esquema del dataset:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [11]:
print("Primeros 5 registros")
df.show(5)

Primeros 5 registros
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  

Para la implementación de esta tarea se elegiran las variables: 
- trip_distance
- passenger_count
- tpep_pickup_datetime
- fare_amount

In [13]:
#----------------------Feature Engineering----------------------------------------
def extract_and_scale_features(row):
    trip_distance, passenger_count, datetime, fare_amount = row
    
    if (trip_distance is None or trip_distance <= 0 or trip_distance >= 100 or
        passenger_count is None or passenger_count <= 0 or passenger_count > 6 or
        datetime is None or
        fare_amount is None or fare_amount <= 0 or fare_amount >= 200):
        return None
    """
    trip_distance <= 0: Viajes inválidos (errores de sensor).
    trip_distance >= 100: Outliers extremos (probablemente errores).
    passenger_count <= 0 or > 6: NYC taxis tienen máximo 5 pasajeros + 1 niño.
    fare_amount <= 0 or >= 200: Errores de medición o fraudes.

    """
    hour_value = float(datetime.hour) #hora del día: 0-23
    day_of_week = float(datetime.weekday() + 1) #dia de la semana: L-D
    
    features = [
        float((trip_distance - 3.0) / 5.0),
        float((passenger_count - 1.5) / 1.0),
        float((hour_value - 12.0) / 7.0),
        float((day_of_week - 4.0) / 2.0)
    ]
    """
    Normalización:  z = (x - μ) / σ, esto por que 
    Convergencia: Gradientes similares en todas las features.
    Velocidad: Adam converge más rápido con datos escalados.
    Estabilidad numérica: Evita overflow/underflow.
    ReLU: Funciona mejor con datos centrados en 0.

    """
    return (features, float(fare_amount)) #Estructura: Tupla (lista_features, label). Compatible con RDD map-reduce y Keras.

print("\nProcesando features")
start = time.time()

rdd_features = df.select( #convierte a rdd
    "trip_distance", "passenger_count", "tpep_pickup_datetime", "fare_amount"
).rdd.map(lambda row: (
    row.trip_distance, row.passenger_count, row.tpep_pickup_datetime, row.fare_amount # Transformación 1-a-1. Ejecución: Lazy (no se ejecuta hasta una acción).
)) #Variables que se eligieron para realizar el proyecto

rdd_scaled = rdd_features \
    .map(extract_and_scale_features) \
    .filter(lambda x: x is not None) \
    .repartition(16) \
    .cache()
# lambda x:x is not None:  Elimina registros None (inválidos)
# .repartition(16): Redistribuir datos en 16 particiones balanceadas
total_scaled = rdd_scaled.count()
print(f"Registros despues de la limpieza: {total_scaled:,} registros. \n Completado en {time.time()-start:.1f}s.")


Procesando features
Registros despues de la limpieza: 2,722,784 registros. 
 Completado en 50.5s.


In [14]:
#----------------------División Train/Test----------------------------------------
train_rdd, test_rdd = rdd_scaled.randomSplit([0.8, 0.2], seed=42) #80% entrenamiento y 20% prueba 

from pyspark import StorageLevel
train_rdd = train_rdd.repartition(16).persist(StorageLevel.MEMORY_AND_DISK)
test_rdd = test_rdd.repartition(8).persist(StorageLevel.MEMORY_AND_DISK)
#StorageLevel.MEMORY_AND_DISK: primer intenta usar memoria, sino cabe utiliza disco

train_count = train_rdd.count()
test_count = test_rdd.count()

print(f"\n Train: {train_count:,} registros")
print(f" Test: {test_count:,} registros")


 Train: 2,177,985 registros
 Test: 544,799 registros


Arquitectura de la red neuronal: 
- capa oculta 1: 64 neuronas, función de activación relu
- capa oculta 2: 32 neuronas, función de activación relu
- capa oculta 3: 16 neuronas, función de activación relu
- capa oculta 4:  8 neuronas, función de activación relu
- capa de salida: 1 neurona , función de activación lineal

- Optimizador: Adam
- Función de costo: MSE
- Metricas adicionales: MAE


In [15]:
#----------------------Modelo-----------------------------------------------------
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
    return model

model = create_model()
print("\n Modelo creado")
model.summary()


 Modelo creado


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,265 (12.75 KB)

 Non-trainable params: 192 (768.00 B)

In [17]:
#----------------------Generador de batches--------------------------------
class RobustRDDBatchGenerator: #Generador que usa toLocalIterator().
    
    def __init__(self, rdd, batch_size=4096, num_batches_per_epoch=None):
        self.rdd = rdd
        self.batch_size = batch_size
        self.total_samples = rdd.count()
        
        if num_batches_per_epoch:
            self.num_batches = num_batches_per_epoch
        else:
            self.num_batches = max(1, self.total_samples // batch_size)
    
    def generate_batches(self, seed=42):
        """
        Genera batches usando toLocalIterator.
        
        toLocalIterator:
        - Itera sobre RDD SIN collect masivo
        - No causa timeout
        - Procesa partición por partición
        - 100% RDD distribuido
        """
        # Sample del RDD
        fraction = min(1.0, (self.batch_size * self.num_batches) / self.total_samples)
        sampled_rdd = self.rdd.sample(False, fraction, seed=seed)
        
        # Usar toLocalIterator
        batch_data = []
        batch_count = 0
        
        
        for item in sampled_rdd.toLocalIterator():
            batch_data.append(item)
            
            # Cuando el batch está lleno, yield
            if len(batch_data) >= self.batch_size:
                X_batch = np.array([x[0] for x in batch_data], dtype=np.float32)
                y_batch = np.array([x[1] for x in batch_data], dtype=np.float32)
                
                yield X_batch, y_batch
                
                batch_data = []
                batch_count += 1
                
                # Limitar número de batches
                if batch_count >= self.num_batches:
                    break
        
        # Último batch parcial
        if batch_data and batch_count < self.num_batches:
            X_batch = np.array([x[0] for x in batch_data], dtype=np.float32)
            y_batch = np.array([x[1] for x in batch_data], dtype=np.float32)
            yield X_batch, y_batch

# Configuración
BATCH_SIZE = 4096  
BATCHES_PER_EPOCH_TRAIN = 400
BATCHES_PER_EPOCH_VAL = 20

train_generator = RobustRDDBatchGenerator(
    train_rdd, 
    batch_size=BATCH_SIZE,
    num_batches_per_epoch=BATCHES_PER_EPOCH_TRAIN
)

test_generator = RobustRDDBatchGenerator(
    test_rdd,
    batch_size=BATCH_SIZE,
    num_batches_per_epoch=BATCHES_PER_EPOCH_VAL
)
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Batches/época train: {BATCHES_PER_EPOCH_TRAIN}")
print(f"  Samples/época: {BATCH_SIZE * BATCHES_PER_EPOCH_TRAIN:,}")

  Batch size: 4096
  Batches/época train: 400
  Samples/época: 1,638,400


In [19]:
#----------------------Entrenamiento----------------------------------------------
print("Entrenamiento.")

EPOCHS = 15

print(f"\n   Configuración:")
print(f"   Épocas: {EPOCHS}")
print(f"   Batches/época: {BATCHES_PER_EPOCH_TRAIN}")
print(f"   Batch size: {BATCH_SIZE}")

history = {'loss': [], 'mae': [], 'val_loss': [], 'val_mae': []}

print("\n  Iniciando el entrenamiento\n")
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    print(f"\nÉpoca {epoch+1}/{EPOCHS}")
    print("-" * 60)
    
    epoch_losses = []
    epoch_maes = []
    
    # Entrenar
    batch_count = 0
    try:
        for X_batch, y_batch in train_generator.generate_batches(seed=epoch):
            metrics = model.train_on_batch(X_batch, y_batch, return_dict=True)
            epoch_losses.append(metrics['loss'])
            epoch_maes.append(metrics['mae'])
            batch_count += 1
            
            if batch_count % 100 == 0:
                print(f"  Batch {batch_count}/{BATCHES_PER_EPOCH_TRAIN} - "
                      f"loss: {np.mean(epoch_losses[-20:]):.4f}")
    except Exception as e:
        print(f"   Error en batch {batch_count}: {e}")
        print(f"  Siguiente epoca")
        continue
    
    if not epoch_losses:
        print("  No se completaron batches, saltando época")
        continue
    
    train_loss = np.mean(epoch_losses)
    train_mae = np.mean(epoch_maes)
    
    # Validación
    val_losses = []
    val_maes = []
    for X_val, y_val in test_generator.generate_batches(seed=epoch):
        val_metrics = model.test_on_batch(X_val, y_val, return_dict=True)
        val_losses.append(val_metrics['loss'])
        val_maes.append(val_metrics['mae'])
    
    val_loss = np.mean(val_losses) if val_losses else train_loss
    val_mae = np.mean(val_maes) if val_maes else train_mae
    
    history['loss'].append(train_loss)
    history['mae'].append(train_mae)
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    
    epoch_time = time.time() - epoch_start
    print(f"\n    Época {epoch+1}:")
    print(f"     loss: {train_loss:.4f} - mae: {train_mae:.4f}")
    print(f"     val_loss: {val_loss:.4f} - val_mae: {val_mae:.4f}")
    print(f"     Tiempo: {epoch_time:.1f}s")
    
    # Early stopping
    if epoch > 3 and val_loss > history['val_loss'][-2]:
        patience = getattr(model, 'patience', 0) + 1
        model.patience = patience
        if patience >= 3:
            print(f"\n   Early stopping")
            break
    else:
        model.patience = 0

training_time = time.time() - start_time

print("\n" + "="*80)
print("✓ Entrenamiento completo")
print("="*80)
print(f"  Tiempo: {training_time/60:.2f} minutos")
print(f"  Épocas: {len(history['loss'])}")
print(f"  Mejor val_loss: {min(history['val_loss']):.4f}")

Entrenamiento.

   Configuración:
   Épocas: 15
   Batches/época: 400
   Batch size: 4096

  Iniciando el entrenamiento


Época 1/15
------------------------------------------------------------
  Batch 100/400 - loss: 110.4774
  Batch 200/400 - loss: 96.9188
  Batch 300/400 - loss: 87.3430
  Batch 400/400 - loss: 80.2727

    Época 1:
     loss: 98.4719 - mae: 5.7322
     val_loss: 78.9331 - val_mae: 4.9447
     Tiempo: 43.5s

Época 2/15
------------------------------------------------------------
  Batch 100/400 - loss: 73.5887
  Batch 200/400 - loss: 69.4125
  Batch 300/400 - loss: 65.9079
  Batch 400/400 - loss: 63.0317

    Época 2:
     loss: 69.5014 - mae: 4.5533
     val_loss: 62.4445 - val_mae: 4.2577
     Tiempo: 42.9s

Época 3/15
------------------------------------------------------------
  Batch 100/400 - loss: 59.9962
  Batch 200/400 - loss: 57.9194
  Batch 300/400 - loss: 56.0852
  Batch 400/400 - loss: 54.4739

    Época 3:
     loss: 57.8943 - mae: 4.0648
     val_loss:

In [20]:
#----------------------Evaluación-------------------------------------------------
print("\nEvaluando modelo...")

eval_generator = RobustRDDBatchGenerator(
    test_rdd,
    batch_size=4096,
    num_batches_per_epoch=100
)

all_predictions = []
all_actuals = []

for X_test, y_test_batch in eval_generator.generate_batches(seed=99):
    y_pred = model.predict(X_test, verbose=0)
    all_predictions.extend(y_pred.flatten().tolist())
    all_actuals.extend(y_test_batch.tolist())

y_test_eval = np.array(all_actuals)
y_pred_eval = np.array(all_predictions)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test_eval, y_pred_eval)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_eval, y_pred_eval)
r2 = r2_score(y_test_eval, y_pred_eval)
mape = np.mean(np.abs((y_test_eval - y_pred_eval) / y_test_eval)) * 100

print("Resultados")
print(f"\n  R²:   {r2:.4f} ({r2*100:.1f}%)")
print(f"  RMSE: ${rmse:.4f}")
print(f"  MAE:  ${mae:.4f}")
print(f"  MAPE: {mape:.2f}%")
print(f"\n  Evaluado en {len(y_test_eval):,} predicciones")


Evaluando modelo...
Resultados

  R²:   0.8837 (88.4%)
  RMSE: $5.6731
  MAE:  $2.6988
  MAPE: 20.65%

  Evaluado en 409,208 predicciones


In [ ]:
#----------------------Guardar----------------------------------------------------
#os.makedirs("modelos", exist_ok=True)
#model_path = f"modelos/taxi_100RDD_ROBUSTO_{datetime.now().strftime('%Y%m%d_%H%M%S')}.h5"
#model.save(model_path)
#print(f"\n✓ Modelo guardado: {model_path}")